In [0]:
# TensorFlow Tutorials
# ML basics with Keras 
# Save and load models
# https://www.tensorflow.org/tutorials/keras/save_and_load

# tfds reshufle: https://www.tensorflow.org/api_docs/python/tf/reshape
# TF.data.Dataset: https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# TF Keras: https://www.tensorflow.org/guide/keras

# Using the SavedModel format: https://www.tensorflow.org/guide/saved_model#save_and_restore_variables
# Writing custome layers and models in keras: https://www.tensorflow.org/guide/keras/custom_layers_and_models
# HDF5: https://en.wikipedia.org/wiki/Hierarchical_Data_Format

# Keras get file: https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file
# pd map: reassign values in a series: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html
# pd get_dummies  to one-hot encode a categorical column: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
# pd pop: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pop.html
# TF Keras EarlyStopping: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
# np expand dimension: https://docs.scipy.org/doc/numpy/reference/generated/numpy.expand_dims.html
# np collapse dimension: https://docs.scipy.org/doc/numpy/reference/generated/numpy.squeeze.html


# https://www.tensorflow.org/tutorials/structured_data/feature_columns
# https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

# mnist tfds dataset: https://www.tensorflow.org/datasets/catalog/mnist

# tfds.load: https://www.tensorflow.org/datasets/api_docs/python/tfds/load
# TF Hub (Transfer Learning): https://www.tensorflow.org/hub

# TF Keras Embedding: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
# TF Hub Keraslayer: https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer
# TF Keras regularizers: https://keras.io/api/layers/regularizers/    https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/Regularizer
# TF Keras Cnv1D: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D
# TF Keras LSTM: https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM
# TF Keras Dense: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
# TF Keras activation functions: https://keras.io/api/layers/activations/

# Live training plots: https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e


# Required to save models in HDF5 format
!pip install -q pyyaml h5py



import pathlib
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from IPython import display
from IPython.display import clear_output
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from datetime import *

# logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
# shutil.rmtree(logdir, ignore_errors=True)

# # Use seaborn for pairplot
# !pip install -q seaborn

# # Use some functions from tensorflow_docs
# !pip install -q git+https://github.com/tensorflow/docs

# import tensorflow_docs as tfdocs
# import tensorflow_docs.plots
# import tensorflow_docs.modeling



print("TF Version: ", tf.__version__)
# print("Eager mode: ", tf.executing_eagerly())
# print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

print('\n')


starttime = datetime.now()


# Download the dataset:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# Take the first 1000 examples:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

print(train_images.shape)

# # reshape(-1) flattens the image data.
train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

# train_images = train_images.take(1000)  #.reshape(-1, 28*28) / 255.0

print(train_images.shape)






et1 = datetime.now() - starttime

print( '\n-> Elapsed execution time: %0.4f seconds.\n' %(et1.total_seconds()) )

2.2.0
TF Version:  2.2.0
GPU is available


(60000, 28, 28)
(1000, 784)

-> Elapsed execution time: 0.3006 seconds.



## Build the model


In [0]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelC

In [0]:
checkpoint_path = "training_1/cp.ckpt"
# os.path.dirname(path) -> Return the directory name of pathname path. This is the first element of the pair returned by passing path to the function split().
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
27/32 [========================>.....] - ETA: 0s - loss: 1.2422 - accuracy: 0.6470
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 1.1564 - accuracy: 0.6740 - val_loss: 0.6927 - val_accuracy: 0.8030
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4333 - accuracy: 0.8750
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.4234 - accuracy: 0.8790 - val_loss: 0.5141 - val_accuracy: 0.8400
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2873 - accuracy: 0.9256
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.2850 - accuracy: 0.9270 - val_loss: 0.4590 - val_accuracy: 0.8570
Epoch 4/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2093 - accuracy: 0.9494
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [0]:
!ls {checkpoint_dir}

print(os.listdir(checkpoint_dir))

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index
['checkpoint', 'cp.ckpt.data-00000-of-00002', 'cp.ckpt.index', 'cp.ckpt.data-00001-of-00002']


Create a new, untrained model. When restoring a model from weights-only, you must have a model with the same architecture as the original model. Since it's the same model architecture, you can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [0]:
# Create a basic untrained model instance of the same architecture as the trained one
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3023 - accuracy: 0.0910
Untrained model, accuracy:  9.10%


Then load the weights from the checkpoint and re-evaluate:

In [0]:
#### !!!Load the weights!!!
model.load_weights(checkpoint_path)

# Re-evaluate the model:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4008 - accuracy: 0.8650
Restored model, accuracy: 86.50%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [0]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)  # weight checkpoint is every 5 epochs

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [0]:
!ls {checkpoint_dir}

print(os.listdir(checkpoint_dir))

checkpoint			  cp-0025.ckpt.data-00001-of-00002
cp-0000.ckpt.data-00000-of-00002  cp-0025.ckpt.index
cp-0000.ckpt.data-00001-of-00002  cp-0030.ckpt.data-00000-of-00002
cp-0000.ckpt.index		  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.data-00001-of-00002  cp-0035.ckpt.data-00000-of-00002
cp-0005.ckpt.index		  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.data-00001-of-00002  cp-0040.ckpt.data-00000-of-00002
cp-0010.ckpt.index		  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.data-00001-of-00002  cp-0045.ckpt.data-00000-of-00002
cp-0015.ckpt.index		  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.data-00001-of-00002  cp-0050.ckpt.data-00000-of-00002
cp-0020.ckpt.index		  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.index
['cp-0040.ckpt.data-0000

## What are these files?

The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

One or more shards that contain your model's weights.
An index file that indicates which weights are stored in a which shard.
If you are only training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001


In [0]:
### choose latest checkpoint:
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# Create a new model instance
model = create_model()

### Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model accuracy:
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

training_2/cp-0050.ckpt
32/32 - 0s - loss: 0.5080 - accuracy: 0.8730
Restored model, accuracy: 87.30%


## Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, `tf.keras`—and `save_weights` in particular—uses the TensorFlow [checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension (saving in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) with a `.h5` extension is covered in the [Save and serialize models](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) guide):

In [0]:
### Save the weights
model.save_weights('./checkpoints/my_checkpoint')  ### save the weights to the path and name specified

# Create a new model instance:
model = create_model()

### Restore the weights:
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model:
loss, acc = model.evaluate(test_images, test_labels, verbose=0)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

Restored model, accuracy: 87.30%


## Save the entire model

Call [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Entire model can be saved in two different file formats (`SavedModel` and `HDF5`). It is to be noted that TensorFlow `SavedModel` format is the default file format in TF2.x. However, model can be saved in `HDF5` format. More details on saving entire model in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/convert/python_api#converting_a_savedmodel_), [HDF5](https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_))

\*Custom objects (e.g. subclassed models or layers) require special attention when saving and loading. See the **Saving custom objects** section below 

### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The [SavedModel guide](https://www.tensorflow.org/guide/saved_model) goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to saving and restoring the model.

In [0]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel:
!mkdir -p saved_model
model.save('saved_model/my_model')  ### Saves the entire model!!!!

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1522 - accuracy: 0.6820
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4182 - accuracy: 0.8910
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2872 - accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2080 - accuracy: 0.9550
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1574 - accuracy: 0.9610
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory:

In [0]:
# my_model directory
!ls saved_model
print(os.listdir('saved_model'))

# Contains an assets folder, saved_model.pb, and variables folder:
!ls saved_model/my_model
print(os.listdir('saved_model//my_model'))

my_model
['my_model']
assets	saved_model.pb	variables
['assets', 'saved_model.pb', 'variables']


Reload a fresh Keras model from the saved model:

In [0]:
### Load a saved model!
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

# Evaluate restored model on test data:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
32/32 - 0s - loss: 0.4824 - accuracy: 0.8370
Restored model, accuracy: 83.70%
(1000, 10)


### HDF5 format

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [0]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.hd5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')


!ls 
print(os.listdir(os.getcwd()))

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1738 - accuracy: 0.6600
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2792 - accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2126 - accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1505 - accuracy: 0.9700
checkpoints  my_model.h5  sample_data  saved_model  training_1	training_2
['.config', 'saved_model', 'training_2', 'checkpoints', 'my_model.h5', 'training_1', 'sample_data']


In [0]:
### Load the hdf5 model:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()


# Evaluate loaded model on test set:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
32/32 - 0s - loss: 0.4524 - accuracy: 0.8490
Restored model, accuracy: 84.90%
(1000, 10)


Keras saves models by inspecting the architecture. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration(what you passed to compile)
* The optimizer and its state, if any (this enables you to restart training where you left)

Keras is not able to save the `v1.x` optimizers (from `tf.compat.v1.train`) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.

### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the orginal code.

To save custom objects to HDF5, you must do the following:

1. Define a `get_config` method in your object, and optionally a `from_config` classmethod.
  * `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
  * `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs (`return cls(**config)`).
2. Pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

See the [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) tutorial for examples of custom objects and `get_config`.
